# Курс Специализация Data Science Дипломный проект. Бриф учебного кейса 
## «Модель прогнозирования стоимости жилья для агентства недвижимости»

#### 3 Этап создания модели

В данном ноутбуке мы сделаем следующее:
* Обработаем и отнормируем признаки
* Построим "наивную" модель, предсказывающую цену по общей площади и городу (с ней будем сравнивать другие модели)
* Построим модель при помощи LinearRegression
* Обучим модель на основе случайного леса RandomForestRegressor
* Обучим модель с L1 и L2 регуляризаций ElasticNetCV
* Сделаем  модель градиентного бустинга CatBoostRegressor
* На основе предыдущих шагов выберем оптимальную модель

In [54]:
import random
import numpy as np 
import pandas as pd 
import sys
import optuna

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold
from sklearn import metrics
from tqdm.notebook import tqdm
from category_encoders import TargetEncoder, CatBoostEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures


# plt
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
#отключим оповещения
import warnings
warnings.filterwarnings("ignore")


In [32]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42
TEST_SIZE = 0.2

In [33]:
df = pd.read_csv('data/data_model.csv')
display(df.head())
df.info()

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,4.0,Southern Pines,2900,28387,NC,418000,False,single_family_home,2019,True,False,False,True,5.2,2.70
1,For Sale,3.0,Spokane Valley,1947,99216,WA,310000,False,single_family_home,2019,False,False,False,False,4.0,1.01
2,Active,2.0,Mason,3588,50401,IA,244900,False,single_family_home,1970,True,True,False,False,3.8,5.60
3,Other,3.0,Houston,1930,77080,TX,311995,False,single_family_home,2019,True,True,True,False,3.0,0.60
4,For Sale,2.0,Flushing,1300,11354,NY,669000,False,condo,1965,False,False,True,False,2.8,0.30


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210669 entries, 0 to 210668
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               210669 non-null  object 
 1   baths                210669 non-null  float64
 2   city                 210669 non-null  object 
 3   sqft                 210669 non-null  int64  
 4   zipcode              210669 non-null  int64  
 5   state                210669 non-null  object 
 6   target               210669 non-null  int64  
 7   pool_encoded         210669 non-null  bool   
 8   Type                 210669 non-null  object 
 9   Year built           210669 non-null  object 
 10  Heating_encoded      210669 non-null  bool   
 11  Cooling_encoded      210669 non-null  bool   
 12  Parking_encoded      210669 non-null  bool   
 13  fireplace_encoded    210669 non-null  bool   
 14  school_rating _mean  210669 non-null  float64
 15  school_dist_min  

In [34]:
# Составим список булевых признаков:
bin_features = ['pool_encoded','Heating_encoded','Cooling_encoded','Parking_encoded','fireplace_encoded']

# Составим список категориальных признаков:
cat_features = ['status','city','zipcode','state','Type','Year built']
 
# Составим список числовых признаков:
num_features = ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']

In [35]:
# подсчет количества уникальных значений в каждой категориальной колонке
for col in cat_features:
   unique_values = df[col].nunique()
   print(f"Количество уникальных значений в категориальной колонке {col}: {unique_values}")

Количество уникальных значений в категориальной колонке status: 12
Количество уникальных значений в категориальной колонке city: 1522
Количество уникальных значений в категориальной колонке zipcode: 3962
Количество уникальных значений в категориальной колонке state: 34
Количество уникальных значений в категориальной колонке Type: 12
Количество уникальных значений в категориальной колонке Year built: 205


In [36]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    # переведем признак зип кода в категориальный
    df_output['zipcode'] = df_output['zipcode'].astype(str)
    # переведем признак год в категориальный
    df_output['Year built'] = df_output['Year built'].astype(str)
    # Нормализация и логорифмирование данных
    #scaler = MinMaxScaler()
    for column in ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']:
        #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        # Логорифмирование
        df_output[column] = df_output[column].apply(lambda x: abs(x))
        constant = 1e-6
        df_output[column] = np.log(df_output[column] + constant)
        
    # ################### Categorical Features ############################################################## 
 
    ohe_status = OneHotEncoder(sparse=False)
    ohe_state = OneHotEncoder(sparse=False)
    ohe_Type = OneHotEncoder(sparse=False)

    status_ohe = ohe_status.fit_transform(df_output['status'].values.reshape(-1,1))
    state_ohe = ohe_state.fit_transform(df_output['state'].values.reshape(-1,1))
    Type_ohe = ohe_Type.fit_transform(df_output['Type'].values.reshape(-1,1))

    le = LabelEncoder()
    state_label = le.fit_transform(df_output['state'])

    year_le = LabelEncoder()
    year_ord = year_le.fit_transform(df_output['Year built'])

    city_le = LabelEncoder()
    city_label = city_le.fit_transform(df_output['city'])

    zip_le = LabelEncoder()
    zip_label = zip_le.fit_transform(df_output['zipcode'])

    # Adding encoded categorical features to the output dataframe
    df_output = df_output.join(pd.DataFrame(status_ohe, columns=['status_' + str(cat) for cat in ohe_status.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(state_ohe, columns=['state_' + str(cat) for cat in ohe_state.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(Type_ohe, columns=['Type_' + str(cat) for cat in ohe_Type.categories_[0]]))
    df_output['state_label'] = state_label
    df_output['year_ord'] = year_ord
    df_output['city_label'] = city_label
    df_output['zip_label'] = zip_label

    # Dropping original categorical columns
    df_output.drop(['status', 'state', 'Type', 'city', 'zipcode','Year built'], axis=1, inplace=True)
    
    return df_output

In [37]:
# Запускаем и проверяем, что получилось
df_encoded = preproc_data(df)
df_encoded.sample(10)

,baths,sqft,target,pool_encoded,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min,...,Type_modern,Type_multi_family_home,Type_other,Type_ranch,Type_single_family_home,Type_townhouse,state_label,year_ord,city_label,zip_label
1455,0.693148,7.151485,12.834415,False,True,True,True,False,0.788458,-1.966106,...,0.0,1.0,0.0,0.0,0.0,0.0,5,117,833,1065
5566,0.693148,7.570959,12.506140,False,True,True,True,False,2.079442,-0.105359,...,0.0,0.0,0.0,0.0,1.0,0.0,28,201,1182,2666
22949,0.693148,6.946976,11.694413,False,True,True,False,False,1.386295,1.163151,...,0.0,0.0,0.0,0.0,0.0,0.0,5,187,1081,1236
177191,1.386295,8.169053,12.959609,False,True,True,True,False,1.667707,0.587787,...,0.0,0.0,0.0,0.0,1.0,0.0,5,175,644,928
173471,0.693148,7.103322,11.849398,False,True,False,True,True,1.163151,-0.798505,...,0.0,0.0,0.0,0.0,1.0,0.0,8,109,242,2072
141116,1.386295,8.114325,12.896717,False,True,True,False,True,2.079442,-0.105359,...,0.0,0.0,0.0,0.0,1.0,0.0,22,139,1025,1598
30939,1.386295,8.241703,12.793581,True,True,True,True,False,1.840550,0.182322,...,0.0,0.0,0.0,0.0,1.0,0.0,28,173,608,2503
77862,0.693148,7.267525,12.100712,False,True,True,False,False,1.308333,-0.916288,...,0.0,0.0,0.0,0.0,0.0,0.0,5,188,692,1299
27792,0.693148,7.327123,12.072541,False,True,True,True,True,1.547563,-0.776527,...,0.0,0.0,0.0,0.0,1.0,0.0,28,165,1182,2635
3968,0.693148,7.494430,12.608165,False,False,False,False,False,2.197225,1.147403,...,0.0,0.0,0.0,0.0,1.0,0.0,5,204,1169,931


In [38]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210669 entries, 0 to 210668
Data columns (total 72 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   baths                    210669 non-null  float64
 1   sqft                     210669 non-null  float64
 2   target                   210669 non-null  float64
 3   pool_encoded             210669 non-null  bool   
 4   Heating_encoded          210669 non-null  bool   
 5   Cooling_encoded          210669 non-null  bool   
 6   Parking_encoded          210669 non-null  bool   
 7   fireplace_encoded        210669 non-null  bool   
 8   school_rating _mean      210669 non-null  float64
 9   school_dist_min          210669 non-null  float64
 10  status_Active            210669 non-null  float64
 11  status_Auction           210669 non-null  float64
 12  status_Back on Market    210669 non-null  float64
 13  status_Coming Soon       210669 non-null  float64
 14  stat

Разделим датасет

In [39]:
y = df_encoded.target.values
X = df_encoded.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

# Обучение модели

## Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю стоимость по общей площади и городу. 
C ней будем сравнивать другие модели.

In [41]:
# Наивная модель
class NaiveModel:
    def __init__(self):
        self.means = None

    def fit(self, X, y):
        X_df = pd.DataFrame(X, columns=['city_label'])
        y_df = pd.DataFrame(y, columns=['target'])
        df = pd.concat([X_df, y_df], axis=1)
        self.means = df.groupby(['city_label'])['target'].mean().reset_index()

    def predict(self, X):
        X = pd.DataFrame(X, columns=['city_label']).copy()
        X['mean'] = np.nan
        for idx, row in self.means.iterrows():
            X.loc[(X['city_label'] == row['city_label']), 'mean'] = row['target']
        
        X['mean'].fillna(X['mean'].mean(), inplace=True)
        return X['mean'].to_numpy()

naive_model = NaiveModel()
naive_model.fit(X_train, y_train)
y_pred_train = naive_model.predict(X_train)
y_pred_test = naive_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")


Train MSE: 0.43
Test MSE: 0.43
Train MAE: 0.48
Test MAE: 0.48
Train R2: -0.00
Test R2: -0.00


MAE (Средняя абсолютная ошибка, Mean Absolute Error) - это мера ошибки, вычисленная как среднее значение абсолютных значений ошибок. Меньшие значения MAE указывают на лучшую точность модели. В данном случае:

Train MAE: 0.48 - это средняя абсолютная ошибка на обучающей выборке.
Test MAE: 0.48 - это средняя абсолютная ошибка на тестовой выборке.
R^2 (коэффициент детерминации) - статистическая мера, которая показывает, насколько хорошо вариации зависимой переменной объясняются моделью. Значения R^2 находятся в диапазоне от -∞ до 1. Чем ближе значение R^2 к 1, тем лучше модель объясняет зависимость между переменными. В данном случае:

Train R2: -0.00 - это коэффициент детерминации на обучающей выборке.
Test R2: -0.00 - это коэффициент детерминации на тестовой выборке.
Проанализировав результаты, можно сделать следующие выводы:

Значения MSE и MAE, как для обучающей, так и для тестовой выборок, практически одинаковы. Это говорит о том, что модель демонстрирует схожую производительность на обеих выборках и не страдает от явного переобучения или недообучения.

Значения R^2 близки к нулю, что говорит о том, что модель слабо объясняет зависимость между переменными. Это может свидетельствовать о низкой предсказательной способности модели.

## Model 2: LinearRegression

In [42]:
# создаём модель линейной регрессии
model = LinearRegression(fit_intercept=False)

# вычисляем коэффициенты регрессии
model.fit(X_train, y_train)

# делаем предсказания с помощью модели
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# вычисляем требуемые метрики
mse_train = metrics.mean_squared_error(y_train, y_train_pred)
mse_test = metrics.mean_squared_error(y_test, y_test_pred)
mae_train = metrics.mean_absolute_error(y_train, y_train_pred)
mae_test = metrics.mean_absolute_error(y_test, y_test_pred)
r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)

# выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.23
Test MSE: 0.23
Train MAE: 0.35
Test MAE: 0.34
Train R2: 0.45
Test R2: 0.45


Среднеквадратическая ошибка (Mean Squared Error, MSE) В данном случае, как для обучающей, так и для тестовой выборки MSE составляет 0.23. Низкие значения MSE указывают на более точные предсказания модели.

Средняя абсолютная ошибка (Mean Absolute Error, MAE) В данном случае MAE составляет 0.35 для обучающей и 0.34 для тестовой выборки. Низкие значения MAE также говорят о хорошей точности предсказания.

Коэффициент детерминации (R2) В данном случае R2 составляет 0.45 как для обучающей, так и для тестовой выборки, что означает, что модель объясняет 45% дисперсии зависимой переменной.

Результаты модели показывают, что она работает с одинаковой точностью на обучающем и тестовом наборе данных, что свидетельствует о том, что модель не переобучилась. Однако, коэффициент детерминации R2 указывает на то, что есть еще большой потенциал для улучшения модели, поскольку ходят возможны другие альтернативы, такие как увеличение размера выборки, использование других переменных или алгоритмов для еще большей предсказательной точности.

## Model 3: RandomForestRegressor

In [43]:
# Создаем экземпляр модели RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=RANDOM_SEED)

# Обучаем модель на обучающих данных
rf_regressor.fit(X_train, y_train)

# Предсказания на обучающих и тестовых данных
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

# Вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.01
Test MSE: 0.07
Train MAE: 0.06
Test MAE: 0.16
Train R2: 0.98
Test R2: 0.84


Среднеквадратическая ошибка (MSE) - В данном случае, значение MSE для обучающей выборки составляет 0.01, тогда как для тестовой выборки - 0.07. Такое различие может указывать на небольшое переобучение модели.

Средняя абсолютная ошибка (MAE) -  В данном случае, значения MAE составили 0.06 для обучающей выборки и 0.16 для тестовой выборки. Разница между этими значениями также может указывать на переобучение.

Коэффициент детерминации (R2) -  В данном случае, R2 равен 0.98 для обучающей выборки и 0.84 для тестовой выборки. Эти значения указывают на то, что модель довольно хорошо работает на обучающей выборке и неплохо предсказывает на тестовой выборке, хотя значение R2 на тестовой выборке заметно ниже, что также может указывать на переобучение модели.

В целом, результаты показывают, что модель имеет хорошую производительность на обучающей выборке, но на тестовой выборке результаты хуже, что может свидетельствовать о переобучении.

## Model 4: ElasticNetCV

In [44]:
# Создаем и тренируем модель ElasticNetCV с кросс-валидацией по 5 фолдам
model_el = ElasticNetCV(cv=5, random_state=RANDOM_SEED)
model_el.fit(X_train, y_train)

# Предсказания для обучающей и тестовой выборок
y_train_pred = model_el.predict(X_train)
y_test_pred = model_el.predict(X_test)

# MSE
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# MAE
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# R2
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.37
Test MSE: 0.38
Train MAE: 0.45
Test MAE: 0.45
Train R2: 0.12
Test R2: 0.11


MSE (Mean Squared Error) - В данном случае обучающая выборка имеет значение 0.37, а тестовая - 0.38. Эти числа близки, что означает, что модель не переобучилась, но в то же время обобщающая способность модели может быть не очень хорошей из-за высоких значений ошибок.

MAE (Mean Absolute Error) - В данном случае средняя абсолютная ошибка составляет 0.45 как для обучающей, так и для тестовой выборки. Это означает, что в среднем модель ошибается на 0.45 при предсказаниях.

R2 (коэффициент детерминации) -  В данном случае R2 равен 0.12 для обучающей выборки и 0.11 для тестовой выборки. Эти значения невысоки, что говорит о плохом качестве модели и низкой способности объяснить изменчивость данных.

Вывод: по представленным результатам видно, что модель имеет среднюю ошибку и невысокий коэффициент детерминации. Это означает, что модель не является оптимальной.

## Model 5: CatBoostRegressor

CatBoostRegressor позволяет работать с необработанными категориальными данными, в отличие от некоторых других алгоритмов машинного обучения, которые требуют предварительной обработки данных, такой как кодирование категориальных переменных в числовые значения.

In [51]:
def log_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    # переведем признак зип кода в категориальный
    df_output['zipcode'] = df_output['zipcode'].astype(str)
    # переведем признак год в категориальный
    df_output['Year built'] = df_output['Year built'].astype(str)
    # Нормализация данных и логорифмирование
    #scaler = MinMaxScaler()
    for column in num_features:
        #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        # Логорифмирование
        df_output[column] = df_output[column].apply(lambda x: abs(x))
        constant = 1e-6
        df_output[column] = np.log(df_output[column] + constant)
    return df_output

In [62]:
# Запускаем и проверяем, что получилось
df_log = log_data(df)
df_log.sample(5)

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
204768,Other,0.693148,Live Oak,7.304516,78233,TX,12.031719,False,single_family_home,1970,True,True,True,True,0.916291,-1.203969
2844,Other,1.098613,San Antonio,7.643962,78221,TX,12.303789,False,single_family_home,2019,True,True,True,True,1.193923,-0.356674
3805,Foreclosure,1.098613,Seattle,7.056175,98198,WA,12.808748,False,single_family_home,1942,True,True,True,True,1.458615,-0.223142
27911,Active,0.693148,Suitland,7.166266,20746,MD,12.611538,False,single_family_home,1935,True,True,True,False,0.788458,-1.203969
160775,For Sale,0.693148,Pt Saint Lucie,7.412160,34984,FL,12.205573,False,single_family_home,1998,True,True,True,False,1.667707,0.943906


In [53]:
y = df_log.target.values
X = df_log.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

In [55]:
param_grid = {
    'iterations': [100, 300, 500],
    'learning_rate': [0.01, 0.03, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [1, 3, 5],
}
cb_model = CatBoostRegressor(random_seed=RANDOM_SEED, silent=True)
grid_search = GridSearchCV(estimator=cb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train, cat_features=cat_features)
best_params = grid_search.best_params_
best_cb_model = CatBoostRegressor(iterations=best_params['iterations'], learning_rate=best_params['learning_rate'], depth=best_params['depth'], l2_leaf_reg=best_params['l2_leaf_reg'], random_seed=RANDOM_SEED, silent=True)
best_cb_model.fit(X_train, y_train, cat_features=cat_features)

y_train_pred = best_cb_model.predict(X_train)
y_test_pred = best_cb_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=  18.2s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=  18.2s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=  18.2s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=  18.6s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.03; total time=  16.3s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.01; total time=  16.6s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.03; total time=  16.3s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.03; total time=  16.4s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.03; total time=  17.0s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_rate=0.03; total time=  16.9s
[CV] END depth=4, iterations=100, l2_leaf_reg=1, learning_

Среднеквадратическая ошибка (MSE):
Train MSE: 0.06 - это значение ошибки на тренировочных данных.
Test MSE: 0.07 - это значение ошибки на тестовых (валидационных) данных.
В данном случае, значения ошибок для обоих наборов данных достаточно близки, что может указывать на отсутствие переобучения.

Средняя абсолютная ошибка (MAE):
Train MAE: 0.17 - это значение MAE на тренировочных данных.
Test MAE: 0.18 - это значение MAE на тестовых данных.
Подобно MSE, MAE для тренировочных и тестовых данных имеет близкие значения, что говорит о хорошей обобщающей способности модели.

Коэффициент детерминации (R2):
Train R2: 0.86 - это R2 на тренировочных данных.
Test R2: 0.83 - это R2 на тестовых данных.
Значения R2 для обеих выборок достаточно высоки, что указывает на неплохую точность модели.

Общий вывод: В целом, модель демонстрирует хорошие показатели точности и обобщающей способности, без признаков переобучения.

In [61]:
data = {'Metric': ['Train MSE', 'Test MSE', 'Train MAE', 'Test MAE', 'Train R2', 'Test R2'],
        'NaiveModel': [0.43, 0.43, 0.48, 0.48, -0.00, -0.00],
        'LinearRegression': [0.23, 0.23, 0.35, 0.34, 0.45, 0.45],
        'RandomForestRegressor': [0.01, 0.07, 0.06, 0.16, 0.98, 0.84],
        'ElasticNetCV': [0.37, 0.38, 0.45, 0.45, 0.12, 0.11],
        'CatBoostRegressor':[0.06, 0.07, 0.17, 0.18, 0.86, 0.83]}

df_metric = pd.DataFrame(data)
df_metric.head(6)


,Metric,NaiveModel,LinearRegression,RandomForestRegressor,ElasticNetCV,CatBoostRegressor
0,Train MSE,0.43,0.23,0.01,0.37,0.06
1,Test MSE,0.43,0.23,0.07,0.38,0.07
2,Train MAE,0.48,0.35,0.06,0.45,0.17
3,Test MAE,0.48,0.34,0.16,0.45,0.18
4,Train R2,-0.00,0.45,0.98,0.12,0.86
5,Test R2,-0.00,0.45,0.84,0.11,0.83


На основе этих результатов, RandomForestRegressor кажется наилучшей моделью, так как она имеет наименьший показатель ошибок MSE и MAE, вместе с высоким коэффициентом детерминации R2 (0.84) на тестовых данных. Однако, стоит учесть переобучение случайного леса, так как разрыв между показателями на тренировочных и тестовых наборах является существенным.

CatBoostRegressor также показывает хорошие результаты и в большинстве случаев является наилучшим, если переобучение RandomForestRegressor считается серьезной проблемой. Ошибки MSE и MAE для CatBoostRegressor на тестовых данных (0.07 и 0.18) несколько хуже, чем у RandomForestRegressor, но разрыв между показателями на тренировочных и тестовых наборах куда меньше, и R2 для тестовых данных всего на 0.01 ниже (0.83 против 0.84).

In [56]:
import pickle

# Сохранение выбранной обученной модели в файл pickle
with open("diplom_web/app/models/best_cb_model.pkl", "wb") as f:
    pickle.dump(best_cb_model, f)

Хотелось бы попробовать обучить модель на полиномиальных признаках но из за большого объема данных крашится ядро.